---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [98]:
import pandas as pd
import numpy as np

def blight_model():
    from sklearn.model_selection import train_test_split
    from sklearn.svm import SVC
    from sklearn.metrics import roc_curve, auc
    from sklearn.model_selection import GridSearchCV
    from sklearn.linear_model import LogisticRegression
    # Your code here
    df = pd.read_csv('train.csv', engine='python')
    columns = ['ticket_id','fine_amount','late_fee','discount_amount',
                   'judgment_amount','disposition','ticket_issued_date','hearing_date',
                   'city','compliance'] # ,'violation_code','grafitti_status','admin_fee','state_fee','clean_up_cost','violation_description'
    df = df[columns]
    df.dropna(subset=['compliance'], inplace=True)
    df=df[df['city']=='Detroit']
    df.set_index('ticket_id', inplace=True)
    X = df.iloc[:,:4]
    X = pd.merge(X,pd.get_dummies(df['disposition']),left_index=True, right_index=True)
    X['tid_dayofweek']=pd.to_datetime(df['ticket_issued_date']).dt.dayofweek
    X['tid_year']=pd.to_datetime(df['ticket_issued_date']).dt.year
    X['tid_month']=pd.to_datetime(df['ticket_issued_date']).dt.month
    X.dropna(axis=1, inplace=True)
    y = df.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    model = LogisticRegression(random_state=0)
    y_score_svm = model.fit(X_train,y_train).decision_function(X_test)
    fpr_svm, tpr_svm, _ = roc_curve(y_test, y_score_svm)
    roc_auc_svm = auc(fpr_svm, tpr_svm)
    
    df_test = pd.read_csv('test.csv', engine='python')
    columns2 = ['ticket_id','fine_amount','late_fee','discount_amount',
                   'judgment_amount','disposition','ticket_issued_date','hearing_date',
                   'city'] # ,'violation_code','grafitti_status','admin_fee','state_fee','clean_up_cost','violation_description'
    df_test = df_test[columns2]
    df_test.set_index('ticket_id', inplace=True)
    X_test2 = df_test.iloc[:,:4]
    temp = pd.get_dummies(df_test['disposition'])
    columns3 = ['Responsible (Fine Waived) by Deter','Responsible by Admission','Responsible by Default', 'Responsible by Determination']
    X_test2 = pd.merge(X_test2,temp[columns3],left_index=True, right_index=True)
    X_test2['tid_dayofweek']=pd.to_datetime(df_test['ticket_issued_date']).dt.dayofweek
    X_test2['tid_year']=pd.to_datetime(df_test['ticket_issued_date']).dt.year
    X_test2['tid_month']=pd.to_datetime(df_test['ticket_issued_date']).dt.month
    X_test2.dropna(axis=1, inplace=True)
    y_prob = model.predict_proba(X_test2)
    df_test['compliance'] = np.float32(y_prob[:,1])
    return df_test['compliance']

In [99]:
blight_model()

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


ticket_id
284932    6.046441e-02
285362    3.193780e-03
285361    8.476895e-02
285338    5.991539e-02
285346    8.476895e-02
285345    5.991539e-02
285347    1.004393e-01
285342    6.750063e-01
285530    3.193780e-03
284989    2.034422e-02
285344    1.004393e-01
285343    3.193780e-03
285340    3.193780e-03
285341    1.004393e-01
285349    8.476895e-02
285348    5.991539e-02
284991    2.034422e-02
285532    2.034422e-02
285406    2.034422e-02
285001    2.015167e-02
285006    3.163029e-03
285405    3.163029e-03
285337    2.093272e-02
285496    1.031009e-01
285497    6.157661e-02
285378    3.287832e-03
285589    2.093272e-02
285585    6.157661e-02
285501    8.705545e-02
285581    3.287832e-03
              ...     
376367    5.000780e-03
376366    3.154761e-02
376362    1.746851e-01
376363    2.032854e-01
376365    5.000780e-03
376364    3.154761e-02
376228    3.154761e-02
376265    3.125242e-02
376286    3.111646e-01
376320    3.095990e-02
376314    3.095990e-02
376327    3.090881e-01
3